In [13]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder



In [2]:
# Load the data
file_path=("Resources/IRL_ClassProjectData_220210.xlsx")
fairtrade_df=pd.read_excel(file_path)

fairtrade_df.head(10)

,Survey Number,Response ID,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,understand_contract,entity_tenure - Months,entity_seasons,...,unsafe_reason_ms_core,accident_last_year,injury_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,leadership_satisfaction_core,factory_recommend_tf_core
0,Survey 2,R_1NsRph9Oqxl6CnG,Farm 1,Man,30,Elementary School,Permanent,Yes,132.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,Yes,No,NaN,7
1,Survey 2,R_2BlpeLG3DymUC2o,Farm 1,Woman,30,No school,Permanent,Yes,108.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,8
2,Survey 2,R_3WMv7jb7gBKEm2t,Farm 1,Man,24,Middle School,Permanent,Yes,67.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,8
3,Survey 2,R_2SBT3CXWOyuor1k,Farm 1,Man,58,Elementary School,Permanent,Yes,79.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,6
4,Survey 2,R_OBEgl4lS6LpSkDL,Farm 1,Man,38,Middle School,Permanent,Yes,108.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,8
5,Survey 2,R_3QY8IjWLk2ZDKDs,Farm 1,Man,37,Elementary School,Permanent,Yes,108.0,NaN,...,NaN,No,NaN,Strongly Agree,Don't know,Strongly disagree,Don't know,No,NaN,8
6,Survey 2,R_1jHPFIUCc8JCpXa,Farm 1,Woman,27,Middle School,Permanent,Yes,33.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,7
7,Survey 2,R_6x0zO9KWBj8KUDL,Farm 1,Man,56,Elementary School,Permanent,Don't know,464.0,NaN,...,NaN,No,NaN,Somewhat agree,Yes,Agree nor disagree,Don't know,No,NaN,5 - Neutral
8,Survey 2,R_cMGCFYIGxSURpzX,Farm 1,Woman,45,No school,Permanent,Yes,84.0,NaN,...,NaN,No,NaN,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,NaN,8
9,Survey 2,R_1q8QVbp49i6YbKB,Farm 1,Woman,28,Elementary School,Permanent,Yes,84.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,Yes,No,NaN,10 - Extremely likely


In [3]:
# Change column names 
fairtrade_df.rename(columns = {'entity_tenure - Months':'entity_tenure_total_months'}, inplace = True)

# # Drop columns 
# fairtrade_df.drop(['Response ID','gender_core','return_next_season_core','age_core','work_last_season_tf','num_week_income_core_usd',
#                    'num_week_income_core_mxn','weeks_per_year','income_change_core','save_money_tf_core',
#                    'financial_resilience_core','unsafe_reason_ms_core','management_complaint_treatment_core',
#                    'leadership_feedback_core','leadership_satisfaction_core','injury_last_year','understand_contract', 'entity_seasons','trust_leadership_core'], axis=1, inplace=True)


# # Drop columns TESTING 
fairtrade_df.drop(['Response ID','work_last_season_tf','num_week_income_core_usd',
                   'num_week_income_core_mxn','weeks_per_year','income_change_core','save_money_tf_core',
                   'financial_resilience_core','unsafe_reason_ms_core','entity_seasons',
                   'leadership_satisfaction_core','injury_last_year','understand_contract','trust_leadership_core'], axis=1, inplace=True)
fairtrade_df.head(10)

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,workplace_satis_core,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_core
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132.0,Somewhat Satisfied,No,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,7
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108.0,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,6
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108.0,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
5,Survey 2,Farm 1,Man,37,Elementary School,Permanent,108.0,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Don't know,Strongly disagree,Don't know,No,8
6,Survey 2,Farm 1,Woman,27,Middle School,Permanent,33.0,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,7
7,Survey 2,Farm 1,Man,56,Elementary School,Permanent,464.0,Somewhat Satisfied,No,Neither good or bad,Somewhat agree,No,Somewhat agree,Yes,Agree nor disagree,Don't know,No,5 - Neutral
8,Survey 2,Farm 1,Woman,45,No school,Permanent,84.0,Very satisfied,No,Somewhat good,Strongly agree,No,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,8
9,Survey 2,Farm 1,Woman,28,Elementary School,Permanent,84.0,Neither satisfied nor dissatisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,10 - Extremely likely


In [4]:
fairtrade_df

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,workplace_satis_core,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_core
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132.0,Somewhat Satisfied,No,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,7
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108.0,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,6
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108.0,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4766,Survey 2,Farm 12,Man,41,College degree or higher,Permanent,240.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,Yes,10 - Extremely likely
4767,Survey 2,Farm 12,Man,36,College degree or higher,Permanent,156.0,Very satisfied,Yes,Very good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,Yes,10 - Extremely likely
4768,Survey 2,Farm 12,Man,37,College degree or higher,Permanent,144.0,Very satisfied,Yes,Very good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,10 - Extremely likely
4769,Survey 2,Farm 12,Woman,22,High School,Permanent,1.0,Somewhat Satisfied,Yes,Very good,Strongly disagree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,10 - Extremely likely


In [5]:
# Drop rows that contain  refused or don't know

values = ["Refused", "Don't know"]
fairtrade_df = fairtrade_df[fairtrade_df.factory_recommend_tf_core.isin(values)==False]
fairtrade_df = fairtrade_df[fairtrade_df.accident_last_year.isin(values)==False]
fairtrade_df = fairtrade_df[fairtrade_df.workplace_satis_core.isin(values)==False]

# Change string values into numbers in factory_recommend_tf_core
dict= {"10 - Extremely likely ": "10", "0- Extremely unlikely": "0", "5 - Neutral": "5"}
fairtrade_df = fairtrade_df.replace({"factory_recommend_tf_core": dict})
fairtrade_df.reset_index(inplace=True, drop=True)

fairtrade_df.head(45)

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,workplace_satis_core,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_core
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132.0,Somewhat Satisfied,No,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,7
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108.0,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,6
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108.0,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
5,Survey 2,Farm 1,Man,37,Elementary School,Permanent,108.0,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Don't know,Strongly disagree,Don't know,No,8
6,Survey 2,Farm 1,Woman,27,Middle School,Permanent,33.0,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,7
7,Survey 2,Farm 1,Man,56,Elementary School,Permanent,464.0,Somewhat Satisfied,No,Neither good or bad,Somewhat agree,No,Somewhat agree,Yes,Agree nor disagree,Don't know,No,5
8,Survey 2,Farm 1,Woman,45,No school,Permanent,84.0,Very satisfied,No,Somewhat good,Strongly agree,No,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,8
9,Survey 2,Farm 1,Woman,28,Elementary School,Permanent,84.0,Neither satisfied nor dissatisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,10


In [ ]:
# fairtrade_df["accident_last_year"].value_counts()
# fairtrade_df["workplace_satis_core"].value_counts()


In [6]:
# Identify the null rows
fairtrade_df.isnull().sum()

Survey Number                            0
ch_account_name                          0
gender_core                              0
age_core                                 0
education_level_core                     0
worker_type_core                         0
entity_tenure_total_months               0
workplace_satis_core                     0
living_wage_tf_core                      0
conditions_rate_core                     0
safety_core                              0
accident_last_year                       0
covid_safety_measures_core             222
return_next_season_core                222
pressure_to_work_core                  222
management_complaint_treatment_core    222
leadership_feedback_core               222
factory_recommend_tf_core              222
dtype: int64

In [7]:
# Drop the null rows 
# fairtrade_df.dropna(subset = ['covid_safety_measures_core','return_next_season_core','pressure_to_work_core','factory_recommend_tf_core'], inplace=True)
# fairtrade_df.isnull().sum()

# DELETE AFTER TESTING

fairtrade_df.dropna(subset = ['covid_safety_measures_core','return_next_season_core','leadership_feedback_core','management_complaint_treatment_core','return_next_season_core','pressure_to_work_core','factory_recommend_tf_core'], inplace=True)
fairtrade_df.isnull().sum()

Survey Number                          0
ch_account_name                        0
gender_core                            0
age_core                               0
education_level_core                   0
worker_type_core                       0
entity_tenure_total_months             0
workplace_satis_core                   0
living_wage_tf_core                    0
conditions_rate_core                   0
safety_core                            0
accident_last_year                     0
covid_safety_measures_core             0
return_next_season_core                0
pressure_to_work_core                  0
management_complaint_treatment_core    0
leadership_feedback_core               0
factory_recommend_tf_core              0
dtype: int64

In [8]:
# Check the datatypes of our columns 
fairtrade_df.dtypes

Survey Number                           object
ch_account_name                         object
gender_core                             object
age_core                                 int64
education_level_core                    object
worker_type_core                        object
entity_tenure_total_months             float64
workplace_satis_core                    object
living_wage_tf_core                     object
conditions_rate_core                    object
safety_core                             object
accident_last_year                      object
covid_safety_measures_core              object
return_next_season_core                 object
pressure_to_work_core                   object
management_complaint_treatment_core     object
leadership_feedback_core                object
factory_recommend_tf_core               object
dtype: object

In [9]:
# Convert columns to numeric data type
fairtrade_df[["entity_tenure_total_months", "factory_recommend_tf_core"]] = fairtrade_df[["entity_tenure_total_months", "factory_recommend_tf_core"]].apply(pd.to_numeric)
fairtrade_df['entity_tenure_total_months'] = fairtrade_df['entity_tenure_total_months'].astype(int)

fairtrade_df.dtypes

Survey Number                          object
ch_account_name                        object
gender_core                            object
age_core                                int64
education_level_core                   object
worker_type_core                       object
entity_tenure_total_months              int64
workplace_satis_core                   object
living_wage_tf_core                    object
conditions_rate_core                   object
safety_core                            object
accident_last_year                     object
covid_safety_measures_core             object
return_next_season_core                object
pressure_to_work_core                  object
management_complaint_treatment_core    object
leadership_feedback_core               object
factory_recommend_tf_core               int64
dtype: object

In [10]:
# Transform factory_recommend_tf_core to have positive or negative status
# Values 0-5 would be coded as '0' (Negative) 
# Values 6-10 would be coded as '1' (Positive)

fairtrade_df['factory_recommend_tf_score'] = pd.cut(fairtrade_df['factory_recommend_tf_core'], bins=[0, 5, 10], include_lowest=True, labels=['0', '1'])

fairtrade_df.head(20)

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,workplace_satis_core,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_core,factory_recommend_tf_score
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132,Somewhat Satisfied,No,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,7,1
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8,1
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8,1
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,6,1
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8,1
5,Survey 2,Farm 1,Man,37,Elementary School,Permanent,108,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Don't know,Strongly disagree,Don't know,No,8,1
6,Survey 2,Farm 1,Woman,27,Middle School,Permanent,33,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,7,1
7,Survey 2,Farm 1,Man,56,Elementary School,Permanent,464,Somewhat Satisfied,No,Neither good or bad,Somewhat agree,No,Somewhat agree,Yes,Agree nor disagree,Don't know,No,5,0
8,Survey 2,Farm 1,Woman,45,No school,Permanent,84,Very satisfied,No,Somewhat good,Strongly agree,No,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,8,1
9,Survey 2,Farm 1,Woman,28,Elementary School,Permanent,84,Neither satisfied nor dissatisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,10,1


In [11]:
# Drop old factory_recommend_tf_core
fairtrade_df.drop(['factory_recommend_tf_core'], axis=1, inplace=True)

fairtrade_df.head(20)

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,workplace_satis_core,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_score
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132,Somewhat Satisfied,No,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,1
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
5,Survey 2,Farm 1,Man,37,Elementary School,Permanent,108,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Don't know,Strongly disagree,Don't know,No,1
6,Survey 2,Farm 1,Woman,27,Middle School,Permanent,33,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
7,Survey 2,Farm 1,Man,56,Elementary School,Permanent,464,Somewhat Satisfied,No,Neither good or bad,Somewhat agree,No,Somewhat agree,Yes,Agree nor disagree,Don't know,No,0
8,Survey 2,Farm 1,Woman,45,No school,Permanent,84,Very satisfied,No,Somewhat good,Strongly agree,No,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,1
9,Survey 2,Farm 1,Woman,28,Elementary School,Permanent,84,Neither satisfied nor dissatisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,1


In [12]:
fairtrade_df.shape

(4430, 18)

# Split the Data into Training and Testing

In [ ]:
# Create our features
colums2drop=['factory_recommend_tf_score','worker_type_core', 'accident_last_year', 'Survey Number','safety_core','living_wage_tf_core','leadership_feedback_core',
             'return_next_season_core']
X=pd.get_dummies(fairtrade_df.drop(colums2drop, axis=1))

# Create our target
y=fairtrade_df['factory_recommend_tf_score']



In [ ]:
X.describe()

In [ ]:
y.describe()

In [ ]:
# Check the value count for y outcomes
# Values 0-6 would be coded as '0' (demoter) 
# Values 7-8 would be coded as '1' (neutral)
# Values 9-10 would be coded as '2' (promoter)
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   random_state=1,
                                                   stratify=y)

X_train.shape

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
clf = BalancedRandomForestClassifier(random_state=0, n_estimators=150)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

# Check the value count for y outcomes
# Values 0-6 would be coded as '0' (demoter) 
# Values 7-8 would be coded as '1' (neutral)
# Values 9-10 would be coded as '2' (promoter)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
feature_importances = clf.feature_importances_
features = sorted(zip(X.columns, clf.feature_importances_), key = lambda x: x[1], reverse=True)
features

In [ ]:
# Export the Dataframe as a new CSV file without the index.
# fairtrade_df.to_csv("fairtrade_clean_data.csv", index=False)